In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interact
import warnings

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define the sector ETFs
sector_etfs = ['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLK', 'XLU', 'XLRE', 'XLB']

# Function to get Quad Witching dates for the past 10 years
def get_quad_witching_dates(years_back=10):
    quad_witching_months = [3, 6, 9, 12]
    today = datetime.date.today()
    current_year = today.year

    quad_witching_dates = []

    for year in range(current_year - years_back, current_year + 1):
        for month in quad_witching_months:
            third_friday = get_third_friday(year, month)
            quad_witching_dates.append(third_friday)

    return [date for date in quad_witching_dates if date < today]

# Function to calculate third Friday of a given month/year
def get_third_friday(year, month):
    first_day = datetime.date(year, month, 1)
    first_friday = first_day + datetime.timedelta(days=(4 - first_day.weekday() + 7) % 7)
    third_friday = first_friday + datetime.timedelta(days=14)
    return third_friday

# Function to get daily ETF data
def get_sector_data(start, end):
    data = yf.download(sector_etfs, start=start, end=end)['Adj Close']
    return data

# Function to calculate cumulative returns
def calculate_cumulative_returns(data, start_date, end_date):
    returns = data.pct_change().loc[start_date:end_date]
    cumulative_returns = (1 + returns).cumprod() - 1
    return cumulative_returns

# Function to filter top and bottom halves based on "Weeks After"
def filter_top_bottom(week_offset, quad_witching_dates, data):
    results = {}
    top_half_wins = 0
    bottom_half_wins = 0
    for i, qd in enumerate(quad_witching_dates[:-1]):
        next_qd = quad_witching_dates[i + 1]
        start_date = qd + datetime.timedelta(weeks=week_offset)
        if start_date > next_qd:  # Skip if "Weeks After" goes beyond the next quad witching date
            continue
        # Get the cumulative return for 1 day or 1 week after the selected week
        initial_returns = data.pct_change().loc[qd:start_date].sum()
        top_half = initial_returns.nlargest(len(initial_returns) // 2).index
        bottom_half = initial_returns.nsmallest(len(initial_returns) // 2).index

        # Calculate cumulative return for both halves until the next quad witching
        top_cumulative = calculate_cumulative_returns(data[top_half], start_date, next_qd)
        bottom_cumulative = calculate_cumulative_returns(data[bottom_half], start_date, next_qd)

        top_cumulative_return = top_cumulative.iloc[-1].mean()
        bottom_cumulative_return = bottom_cumulative.iloc[-1].mean()

        results[qd] = {'top_half': top_cumulative_return, 'bottom_half': bottom_cumulative_return}
        
        # Track which half performs better
        if top_cumulative_return > bottom_cumulative_return:
            top_half_wins += 1
        else:
            bottom_half_wins += 1

    total = len(results)
    top_half_percentage = (top_half_wins / total) * 100 if total > 0 else 0
    bottom_half_percentage = (bottom_half_wins / total) * 100 if total > 0 else 0

    return results, top_half_percentage, bottom_half_percentage

# Plotting the results
def plot_results(results, top_half_percentage, bottom_half_percentage):
    quad_witching_dates = list(results.keys())
    top_returns = [results[date]['top_half'] for date in quad_witching_dates]
    bottom_returns = [results[date]['bottom_half'] for date in quad_witching_dates]

    width = 15  # Width of the bars in days
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create bars for top half and bottom half, offset by a small margin for visibility
    ax.bar([d - datetime.timedelta(days=width//2) for d in quad_witching_dates], top_returns, width=width, label='Top Half', color='blue', alpha=0.7)
    ax.bar([d + datetime.timedelta(days=width//2) for d in quad_witching_dates], bottom_returns, width=width, label='Bottom Half', color='orange', alpha=0.7)

    # Adding labels and gridlines
    ax.set_xlabel("Quad Witching Dates")
    ax.set_ylabel("Cumulative Returns")
    ax.set_title("Performance of Top vs Bottom ETFs After Quad Witching")
    ax.legend()

    # Adding gridlines for better readability
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Display percentages
    ax.text(0.02, 0.92, f'Top Half Outperforms: {top_half_percentage:.2f}% of the time', transform=fig.transFigure, fontsize=10, color='green')
    ax.text(0.02, 0.88, f'Bottom Half Outperforms: {bottom_half_percentage:.2f}% of the time', transform=fig.transFigure, fontsize=10, color='red')

    plt.show()

# Interactive widget to select "Weeks After"
def on_week_selected(week):
    quad_witching_dates = get_quad_witching_dates(years_back=10)

    # Download the data for the past 10 years
    data = get_sector_data(start=quad_witching_dates[0] - datetime.timedelta(days=30), end=datetime.date.today())

    # Calculate top/bottom halves and cumulative returns
    results, top_half_percentage, bottom_half_percentage = filter_top_bottom(week, quad_witching_dates, data)

    # Plot the results
    plot_results(results, top_half_percentage, bottom_half_percentage)

# Create the dropdown widget
dropdown = Dropdown(
    options=[(f"{i} Weeks After", i) for i in range(13)],
    description='Weeks After',
)

# Set up interaction
interact(on_week_selected, week=dropdown)
